При реализации кода данного ноутбука руководствовалась этим учебником по тензорному потоку:              
https://www.tensorflow.org/tutorials/structured_data/time_series#normalize_the_data

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
# !pip install --upgrade pip
# !pip uninstall tensorflow
# !pip install -U tensorflow==2.3.2

In [3]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
print(tf.__version__)

2.6.2


В данных 54 магазина и 33 семейства продуктов.                     
Временной ряд тренировочных данных с 01.01.2013 по 15.08.2017.              
Временной ряд тестовых данных для submition составляет 16 дней после последней даты тренировочных данных: с 16.08.2017 по 31.08.2017.                   
                                 
Нужно составить прогноз продаж для каждого из семейств продуктов в каждом из магазинов.     
                             
В отдельном ноутбуке проведен Feature engineering и сформированы необходимые датасеты.
                                
**Текущие датасеты**                   
1. final_featured_data - объединенные данные test и train с новыми признаками, отобранными по значимости.
2. zero_prediction - данные тех товаров, которые не продавались в конкретном магазине с начала 2013 года, исходя из чего можно предположить, что данные товары не будут продаваться в ближайшие 16 дней. Этот датасет мы будем объединять с предсказанными данными перед отправкой в submit.           
3. fuller_not_stat - не стационарные по результатам подсчета критерия Дикки-Фуллера ряды из числа рядов "магазин+семейство товаров"

In [5]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import gc
from tqdm.auto import tqdm
import datetime as dt
pd.set_option('display.max_columns', None)

# Import Data

In [6]:
# Import
data = pd.read_csv("../input/featured-data-with-imp-feat/final_featured_data.csv")
zero_prediction = pd.read_csv("../input/zero-prediction/zero_prediction.csv")
fuller_result = pd.read_csv("../input/fuller-result/fuller_not_stat.csv")

# Datetime
data["date"] = pd.to_datetime(data.date)

zero_prediction = zero_prediction.set_index(['store_nbr', 'family', 'date']).sort_index()

In [7]:
data.sample()

,family,store_nbr,date,id,sales,season,quarter,week_of_month,year,day_of_week,month,day_of_month,onpromotion,week_of_year,day_of_year
2330846,POULTRY,34,2016-11-08,2502847,117.006996,3,4,2,2016,1,11,8,0.0,45,313


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2727160 entries, 0 to 2727159
Data columns (total 15 columns):
 #   Column         Dtype         
---  ------         -----         
 0   family         object        
 1   store_nbr      int64         
 2   date           datetime64[ns]
 3   id             int64         
 4   sales          float64       
 5   season         int64         
 6   quarter        int64         
 7   week_of_month  int64         
 8   year           int64         
 9   day_of_week    int64         
 10  month          int64         
 11  day_of_month   int64         
 12  onpromotion    float64       
 13  week_of_year   int64         
 14  day_of_year    int64         
dtypes: datetime64[ns](1), float64(2), int64(11), object(1)
memory usage: 312.1+ MB


# Метрики

In [9]:
def compute_metrics(real, forecast):
    result = {}
    real=np.array(real)
    forecast=np.array(forecast)
    result['MSE'] = round(((real-forecast)**2).mean(),4)
    result['RMSE'] = round((((real-forecast)**2)**(1/2)).mean(),4)
    mape_list = []
    smap_list = []
    for i in range(len(real)):
        if real[i] == 0:
            mape_list.append(0)
            smap_list.append(0)
        else:
            mape_list.append(abs(real[i]-forecast[i])/real[i])
            smap_list.append(2.0 * np.mean(np.abs(forecast[i] - real[i]) / (np.abs(forecast[i]) + np.abs(real[i]))))
        
    result['MAPE'] = round(np.mean(mape_list),4)
    result['SMAP'] = round(np.mean(smap_list),4)                   
    #result['SMAP'] = round(2.0 * np.mean(np.abs(forecast - real) / (np.abs(forecast) + np.abs(real))),4)
    return pd.Series(result)

# Parameters

In [10]:
input_width = 1*7
label_width = 16

MAX_EPOCHS = 50
learning_rate = 0.001

# Validation

Преобразуем наш датасет так, чтобы каждый столбец являлся временным рядом по комбинации магазин~семейство товаров.                                        
**Multi-output**

In [11]:
# Reshape data
df = data.assign(key=data['store_nbr'].astype('str') + '~' + data['family'])
df = pd.pivot_table(df, values = 'sales', index=['date'], columns='key').reset_index()
date_time = df.date
df = df.drop('date', axis=1)
column_indices = {name: i for i, name in enumerate(df.columns)}
df.head(3)

key,10~AUTOMOTIVE,10~BABY CARE,10~BEAUTY,10~BEVERAGES,10~BREAD/BAKERY,10~CELEBRATION,10~CLEANING,10~DAIRY,10~DELI,10~EGGS,10~FROZEN FOODS,10~GROCERY I,10~GROCERY II,10~HARDWARE,10~HOME AND KITCHEN I,10~HOME AND KITCHEN II,10~HOME APPLIANCES,10~HOME CARE,10~LADIESWEAR,10~LAWN AND GARDEN,10~LINGERIE,"10~LIQUOR,WINE,BEER",10~MAGAZINES,10~MEATS,10~PERSONAL CARE,10~PET SUPPLIES,10~PLAYERS AND ELECTRONICS,10~POULTRY,10~PREPARED FOODS,10~PRODUCE,10~SCHOOL AND OFFICE SUPPLIES,10~SEAFOOD,11~AUTOMOTIVE,11~BABY CARE,11~BEAUTY,11~BEVERAGES,11~BREAD/BAKERY,11~CELEBRATION,11~CLEANING,11~DAIRY,11~DELI,11~EGGS,11~FROZEN FOODS,11~GROCERY I,11~GROCERY II,11~HARDWARE,11~HOME AND KITCHEN I,11~HOME AND KITCHEN II,11~HOME APPLIANCES,11~HOME CARE,11~LADIESWEAR,11~LAWN AND GARDEN,11~LINGERIE,"11~LIQUOR,WINE,BEER",11~MAGAZINES,11~MEATS,11~PERSONAL CARE,11~PET SUPPLIES,11~PLAYERS AND ELECTRONICS,11~POULTRY,11~PREPARED FOODS,11~PRODUCE,11~SCHOOL AND OFFICE SUPPLIES,11~SEAFOOD,12~AUTOMOTIVE,12~BABY CARE,12~BEAUTY,12~BEVERAGES,12~BREAD/BAKERY,12~CELEBRATION,12~CLEANING,12~DAIRY,12~DELI,12~EGGS,12~FROZEN FOODS,12~GROCERY I,12~GROCERY II,12~HARDWARE,12~HOME AND KITCHEN I,12~HOME AND KITCHEN II,12~HOME APPLIANCES,12~HOME CARE,12~LADIESWEAR,12~LAWN AND GARDEN,12~LINGERIE,"12~LIQUOR,WINE,BEER",12~MAGAZINES,12~MEATS,12~PERSONAL CARE,12~PET SUPPLIES,12~PLAYERS AND ELECTRONICS,12~POULTRY,12~PREPARED FOODS,12~PRODUCE,12~SCHOOL AND OFFICE SUPPLIES,12~SEAFOOD,13~AUTOMOTIVE,13~BEAUTY,13~BEVERAGES,13~BREAD/BAKERY,13~CELEBRATION,13~CLEANING,13~DAIRY,13~DELI,13~EGGS,13~FROZEN FOODS,13~GROCERY I,13~GROCERY II,13~HARDWARE,13~HOME AND KITCHEN I,13~HOME AND KITCHEN II,13~HOME APPLIANCES,13~HOME CARE,13~LADIESWEAR,13~LAWN AND GARDEN,13~LINGERIE,"13~LIQUOR,WINE,BEER",13~MAGAZINES,13~MEATS,13~PERSONAL CARE,13~PET SUPPLIES,13~PLAYERS AND ELECTRONICS,13~POULTRY,13~PREPARED FOODS,13~PRODUCE,13~SCHOOL AND OFFICE SUPPLIES,13~SEAFOOD,14~AUTOMOTIVE,14~BABY CARE,14~BEAUTY,14~BEVERAGES,14~BREAD/BAKERY,14~CELEBRATION,14~CLEANING,14~DAIRY,14~DELI,14~EGGS,14~FROZEN FOODS,14~GROCERY I,14~GROCERY II,14~HARDWARE,14~HOME AND KITCHEN I,14~HOME AND KITCHEN II,14~HOME APPLIANCES,14~HOME CARE,14~LADIESWEAR,14~LINGERIE,"14~LIQUOR,WINE,BEER",14~MAGAZINES,14~MEATS,14~PERSONAL CARE,14~PET SUPPLIES,14~PLAYERS AND ELECTRONICS,14~POULTRY,14~PREPARED FOODS,14~PRODUCE,14~SCHOOL AND OFFICE SUPPLIES,14~SEAFOOD,15~AUTOMOTIVE,15~BABY CARE,15~BEAUTY,15~BEVERAGES,15~BREAD/BAKERY,15~CELEBRATION,15~CLEANING,15~DAIRY,15~DELI,15~EGGS,15~FROZEN FOODS,15~GROCERY I,15~GROCERY II,15~HARDWARE,15~HOME AND KITCHEN I,15~HOME AND KITCHEN II,15~HOME APPLIANCES,15~HOME CARE,15~LADIESWEAR,15~LAWN AND GARDEN,15~LINGERIE,"15~LIQUOR,WINE,BEER",15~MAGAZINES,15~MEATS,15~PERSONAL CARE,15~PET SUPPLIES,15~PLAYERS AND ELECTRONICS,15~POULTRY,15~PREPARED FOODS,15~PRODUCE,15~SCHOOL AND OFFICE SUPPLIES,15~SEAFOOD,16~AUTOMOTIVE,16~BABY CARE,16~BEAUTY,16~BEVERAGES,16~BREAD/BAKERY,16~CELEBRATION,16~CLEANING,16~DAIRY,16~DELI,16~EGGS,16~FROZEN FOODS,16~GROCERY I,16~GROCERY II,16~HARDWARE,16~HOME AND KITCHEN I,16~HOME AND KITCHEN II,16~HOME APPLIANCES,16~HOME CARE,16~LAWN AND GARDEN,16~LINGERIE,"16~LIQUOR,WINE,BEER",16~MAGAZINES,16~MEATS,16~PERSONAL CARE,16~PET SUPPLIES,16~PLAYERS AND ELECTRONICS,16~POULTRY,16~PREPARED FOODS,16~PRODUCE,16~SCHOOL AND OFFICE SUPPLIES,16~SEAFOOD,17~AUTOMOTIVE,17~BABY CARE,17~BEAUTY,17~BEVERAGES,17~BREAD/BAKERY,17~CELEBRATION,17~CLEANING,17~DAIRY,17~DELI,17~EGGS,17~FROZEN FOODS,17~GROCERY I,17~GROCERY II,17~HARDWARE,17~HOME AND KITCHEN I,17~HOME AND KITCHEN II,17~HOME APPLIANCES,17~HOME CARE,17~LADIESWEAR,17~LAWN AND GARDEN,17~LINGERIE,"17~LIQUOR,WINE,BEER",17~MAGAZINES,17~MEATS,17~PERSONAL CARE,17~PET SUPPLIES,17~PLAYERS AND ELECTRONICS,17~POULTRY,17~PREPARED FOODS,17~PRODUCE,17~SCHOOL AND OFFICE SUPPLIES,17~SEAFOOD,18~AUTOMOTIVE,18~BABY CARE,18~BEAUTY,18~BEVERAGES,18~BREAD/BAKERY,18~CELEBRATION,18~CLEANING,18~DAIRY,18~DELI,18~EGGS,18~FROZEN FOODS,18~GROCERY I,18~GROCERY II,18~HARDWARE,18~HOME AND KITCHEN I,18~HOME

In [12]:
num_features = df.shape[1]
test_df = df[-label_width:]
val_df = df[-(2*label_width):-label_width]
train_df = df[:-(2*label_width)]

# Normalization

In [13]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
train_df = train_df.fillna(0)
val_df = (val_df - train_mean) / train_std
val_df = val_df.fillna(0)
test_df = (test_df - train_mean) / train_std
test_df = test_df.fillna(0)

train_df[train_df == np.inf] = 0
val_df[val_df == np.inf] = 0
test_df[test_df == np.inf] = 0

# Data windowing

In [14]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

In [15]:
def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window

In [16]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.train))
        # And cache it for next time
        self._example = result
    return result

def compile_and_fit(model, window, MAX_EPOCHS=20, learning_rate=0.2, patience=10):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
                metrics=[tf.metrics.MeanAbsoluteError()])
    
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    
    return history

# LSTM

In [17]:
WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
# WindowGenerator.make_dataset = make_dataset
#WindowGenerator.split_window = split_window

In [18]:
# Generate windows for training batches
window = WindowGenerator(input_width=input_width, label_width=label_width, shift=label_width,
                         train_df=train_df.reset_index(), val_df=val_df.reset_index(), test_df=test_df.reset_index())

In [19]:
print("Train set size: (%s, %s)" %(train_df.shape[0], train_df.shape[1]))
print("Validation set size: (%s, %s)"  %(test_df.shape[0], test_df.shape[1]))
print("Test set size: (%s, %s) \n"  %(val_df.shape[0], val_df.shape[1]))

# Generate windows for training batches
window = WindowGenerator(input_width=input_width, label_width=label_width, shift=label_width,
                         train_df=train_df, val_df=val_df, test_df=test_df)

model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    tf.keras.layers.Conv1D(filters=128, kernel_size=(input_width,), activation='relu'),
    tf.keras.layers.LSTM(128, return_sequences=False),
    tf.keras.layers.Dense(label_width*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    tf.keras.layers.Reshape([label_width, num_features])
])

print('Input shape:', window.example[0].shape)
print('Labels shape:', window.example[1].shape)
print('Output shape:', model(window.example[0]).shape)

history = compile_and_fit(model, window, MAX_EPOCHS, learning_rate)

Train set size: (1652, 1729)
Validation set size: (16, 1729)
Test set size: (16, 1729) 



2022-02-27 21:19:43.432819: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-02-27 21:19:43.730657: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Input shape: (32, 7, 1729)
Labels shape: (32, 16, 1729)
Output shape: (32, 16, 1729)
Epoch 1/50
51/51 [==============================] - 5s 40ms/step - loss: 0.7587 - mean_absolute_error: 0.5534
Epoch 2/50
51/51 [==============================] - 2s 46ms/step - loss: 0.6434 - mean_absolute_error: 0.4886
Epoch 3/50
51/51 [==============================] - 2s 47ms/step - loss: 0.5981 - mean_absolute_error: 0.4604
Epoch 4/50
51/51 [==============================] - 3s 58ms/step - loss: 0.5703 - mean_absolute_error: 0.4454
Epoch 5/50
51/51 [==============================] - 3s 64ms/step - loss: 0.5532 - mean_absolute_error: 0.4364
Epoch 6/50
51/51 [==============================] - 2s 40ms/step - loss: 0.5403 - mean_absolute_error: 0.4306
Epoch 7/50
51/51 [==============================] - 2s 41ms/step - loss: 0.5305 - mean_absolute_error: 0.4259
Epoch 8/50
51/51 [==============================] - 3s 51ms/step - loss: 0.5224 - mean_absolute_error: 0.4223
Epoch 9/50
51/51 [=================

In [20]:
y_hat = model.predict(val_df.values[-input_width:, :].reshape(1, input_width, num_features))
predict_df = (pd.DataFrame(y_hat.reshape(label_width, num_features), columns=df.columns)*train_std + train_mean).assign(date=date_time[-label_width:].values)
columns_to_keep = [e for e in predict_df.columns if '~' in e or e == 'date'] 
predict_df = predict_df[columns_to_keep]
predict_df = predict_df.melt(id_vars =['date'], value_vars =[c for c in predict_df.columns if c != 'date'])
predict_df[['store_nbr', 'family']] = predict_df.key.str.split('~', expand=True)
predict_df['store_nbr'] = predict_df.store_nbr.astype('int')
predict_df.drop('key', axis=1, inplace=True)
predict_df = pd.merge(data[(data.date > (data.date.max()-dt.timedelta(days=2 * label_width))) & (data.date < (data.date.max()-dt.timedelta(days=label_width)))].drop('id', axis=1), predict_df, on=['date', 'store_nbr', 'family'], how='left')
predict_df['value'] = np.clip(predict_df.value, 0, np.inf)
predict_df.tail()

,family,store_nbr,date,sales,season,quarter,week_of_month,year,day_of_week,month,day_of_month,onpromotion,week_of_year,day_of_year,value
25930,SEAFOOD,54,2017-08-10,2.0,2,3,2,2017,3,8,10,0.0,32,222,2.467032
25931,SEAFOOD,54,2017-08-11,0.0,2,3,2,2017,4,8,11,0.0,32,223,2.072820
25932,SEAFOOD,54,2017-08-12,1.0,2,3,2,2017,5,8,12,1.0,32,224,3.269373
25933,SEAFOOD,54,2017-08-13,2.0,2,3,2,2017,6,8,13,0.0,32,225,3.978170
25934,SEAFOOD,54,2017-08-14,0.0,2,3,2,2017,0,8,14,0.0,33,226,3.498869


In [21]:
compute_metrics(predict_df.sales, predict_df.value)

MSE     135600.2033
RMSE       103.6198
MAPE         0.3877
SMAP         0.3168
dtype: float64

## Submition

In [22]:
y_hat = model.predict(df.values[-input_width:, :].reshape(1, input_width, num_features))
predict_df = (pd.DataFrame(y_hat.reshape(label_width, num_features), columns=df.columns)*train_std + train_mean).assign(date=pd.date_range(date_time.iloc[-1] + timedelta(days=1), date_time.iloc[-1] + timedelta(days=label_width)))

predict_df = predict_df.melt(id_vars =['date'], value_vars =[c for c in predict_df.columns if c != 'date'])
predict_df[['store_nbr', 'family']] = predict_df.key.str.split('~', expand=True)
predict_df = predict_df.rename(columns={'value': 'y_hat'})
predict_df['store_nbr'] = predict_df.store_nbr.astype('int')
predict_df.drop('key', axis=1, inplace=True)

predict_df = pd.merge(data[data.date > (data.date.max()-dt.timedelta(days=label_width))], predict_df,  on=['date', 'store_nbr', 'family'], how='left')
predict_df['y_hat'] = predict_df.y_hat.clip(0)
predict_df = predict_df.set_index(['store_nbr', 'family', 'date']).sort_index()
for i in zero_prediction.index:
    predict_df.loc[i,'y_hat']=0
predict_df = predict_df.reset_index()

predict_df[['id', 'y_hat']].rename(columns={'y_hat': 'sales'}).to_csv('nn1_submission.csv', index=False)

**Результат на kaggle 0.77364**